# Routing to IO

## Routing electrical
For routing low speed DC electrical ports you can use sharp corners instead of smooth bends.

You can also define `port.orientation = None` to ignore the port orientation for low speed DC ports.

For single route between ports you can use `route_single_electrical`

### route_single_electrical


`route_single_electrical` has `bend = wire_corner` with a 90deg bend corner.

In [ ]:
from functools import partial

import gdsfactory as gf
from gdsfactory.generic_tech import get_generic_pdk
from gdsfactory.samples.big_device import big_device

gf.config.rich_output()

c = gf.Component()
pt = c << gf.components.pad_array(port_orientation=270, columns=3)
pb = c << gf.components.pad_array(port_orientation=90, columns=3)
pt.dmove((70, 200))
c.plot()

In [ ]:
c = gf.Component()
pt = c << gf.components.pad_array(port_orientation=270, columns=3)
pb = c << gf.components.pad_array(port_orientation=90, columns=3)
pt.dmove((70, 200))
route = gf.routing.route_single_electrical(c, pt.ports["e11"], pb.ports["e11"])
c.plot()

In [ ]:
c = gf.Component()
pt = c << gf.components.pad_array(orientation=None, columns=3)
pb = c << gf.components.pad_array(orientation=None, columns=3)
pt.dmove((70, 200))
route = gf.routing.route_single(
    c, pt.ports["e11"], pb.ports["e11"], cross_section="strip"
)
c.plot()

There is also `bend = wire_corner45` for 45deg bend corner with parametrizable "radius":

In [ ]:
c = gf.Component()
pt = c << gf.components.pad_array(port_orientation=270, columns=1)
pb = c << gf.components.pad_array(port_orientation=90, columns=1)
pt.dmove((300, 300))
route = gf.routing.route_single(
    c,
    pt.ports["e11"],
    pb.ports["e11"],
    bend="wire_corner45",
    port_type="electrical",
    taper=None,
    cross_section="metal_routing",
    allow_width_mismatch=True,
)
c.plot()

In [ ]:
c = gf.Component()
pt = c << gf.components.pad_array(orientation=270, columns=1)
pb = c << gf.components.pad_array(orientation=90, columns=1)
pt.dmove((300, 300))
route = gf.routing.route_single(
    c,
    pt.ports["e11"],
    pb.ports["e11"],
    bend="wire_corner45",
    radius=100,
    taper=None,
    cross_section="metal_routing",
    port_type="electrical",
    allow_width_mismatch=True,
)
c.plot()

### route_quad

In [ ]:
c = gf.Component()
pt = c << gf.components.pad_array(orientation=270, columns=3)
pb = c << gf.components.pad_array(orientation=90, columns=3)
pt.dmove((100, 200))
gf.routing.route_quad(c, pt.ports["e11"], pb.ports["e11"], layer=(49, 0))
c.plot()

### route_single_from_steps

In [ ]:
import gdsfactory as gf

c = gf.Component()
pt = c << gf.components.pad_array(orientation=270, columns=3)
pb = c << gf.components.pad_array(orientation=90, columns=3)
pt.dmove((100, 200))
route = gf.routing.route_single_from_steps(
    c,
    pb.ports["e11"],
    pt.ports["e11"],
    steps=[
        {"y": 200},
    ],
    cross_section="metal_routing",
    bend=gf.components.wire_corner,
    port_type="electrical",
    allow_width_mismatch=True,
)
c.plot()

In [ ]:
c = gf.Component()
pt = c << gf.components.pad_array(orientation=None, columns=3)
pb = c << gf.components.pad_array(orientation=None, columns=3)
pt.dmove((100, 200))
route = gf.routing.route_single_from_steps(
    c,
    pb.ports["e11"],
    pt.ports["e11"],
    steps=[
        {"y": 200},
    ],
    cross_section="metal_routing",
    bend=gf.components.wire_corner,
    port_type="electrical",
    allow_width_mismatch=True,
)
c.plot()

### route_bundle_electrical

For routing groups of ports you can use `route_bundle` that returns a bundle of routes using a bundle router (also known as bus or river router)

In [ ]:
c = gf.Component()
pt = c << gf.components.pad_array(orientation=270, columns=3)
pb = c << gf.components.pad_array(orientation=90, columns=3)
pt.dmove((100, 200))

routes = gf.routing.route_bundle_electrical(
    c, pt.ports, pb.ports, end_straight_length=60, separation=30
)
c.plot()

## Routing to pads

You can also route to electrical pads.

In [ ]:
c = gf.components.straight_heater_metal(length=100.0)
cc = gf.routing.add_pads_bot(component=c, port_names=("l_e4", "r_e4"), fanout_length=50)
cc.plot()

In [ ]:
c = gf.components.straight_heater_metal(length=100.0)
cc = gf.routing.add_pads_bot(component=c, port_names=("l_e4", "r_e4"), fanout_length=50)
cc.plot()

In [ ]:
c = gf.components.straight_heater_metal(length=110)
cc = gf.routing.add_pads_top(component=c, port_names=("l_e2", "r_e2"), fanout_length=50)
cc.plot()

In [ ]:
c = gf.c.nxn(
    xsize=600,
    ysize=200,
    north=2,
    south=3,
    wg_width=10,
    layer="M3",
    port_type="electrical",
)
cc = gf.routing.add_pads_top(component=c)
cc.plot()

In [ ]:
n = west = north = south = east = 10
spacing = 20
c = gf.components.nxn(
    xsize=n * spacing,
    ysize=n * spacing,
    west=west,
    east=east,
    north=north,
    south=south,
    port_type="electrical",
    wg_width=10,
)
c.plot()

In [ ]:
cc = gf.routing.add_pads_top(component=c, fanout_length=300)
cc.plot()

## Routing to optical terminations

### Route to Fiber Array

You can route to a fiber array.

In [ ]:
from gdsfactory.samples.big_device import big_device

component = big_device(nports=10)
c = gf.routing.add_fiber_array(component=component, radius=10.0, fanout_length=60.0)
c.plot()

You can also mix and match TE and TM grating couplers. Notice that the `TM` polarization grating coupler is bigger.

In [ ]:
import gdsfactory as gf

c = gf.components.mzi_phase_shifter()
gcte = gf.components.grating_coupler_te

cc = gf.routing.add_fiber_array(
    component=c,
    optical_routing_type=2,
    grating_coupler=gf.components.grating_coupler_te,
    radius=20,
)
cc.plot()

### Route to edge couplers

You can also route Edge couplers to a fiber array or to both sides of the chip.

For routing to both sides you can follow different strategies:

1. Place the edge couplers and route your components to the edge couplers.
2. Extend your component ports to each side.
3. Anything you imagine ...